# compare time of different read data methods

In [1]:
import numpy as np
import pandas as pd
import pptk
import datetime
import torch

## use numpy ndarray read data

In [4]:
file = "./data/arch/Train/1_TR_cloister.txt"
time3 = datetime.datetime.now()
data = np.loadtxt(file)
scene_points = data[:,0:3].astype('float32')
segment_label = data[:,6].astype('int64')
time4 = datetime.datetime.now()
print(scene_points.shape)
print(segment_label.shape)
print(time4-time3)

(15740229, 3)
(15740229,)
0:03:34.670069


In [19]:
import numpy as np
choice = np.random.choice(len(segment_label), 1024, replace=True)
point_set = scene_points[choice, :]

NameError: name 'segment_label' is not defined

## use numpy read file + use torch read data

In [3]:
file = "./data/arch/Train/1_TR_cloister.txt"
time1 = datetime.datetime.now()
data = np.loadtxt(file)
scene_points = torch.from_numpy(data[:, 0:3].astype('float32'))
segment_label = torch.from_numpy(data[:,6].astype('int32'))
time2 = datetime.datetime.now()
print(scene_points.shape)
print(segment_label.shape)
print(time2-time1)

torch.Size([15740229, 3])
torch.Size([15740229])
0:03:52.921357


In [2]:
import os
print(os.getcwd())

/home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud


In [3]:
!cd datasets

In [4]:
print(os.getcwd())

/home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud


In [5]:
!ls

 common.py		   LOG		 shapenetcorev2_hdf5_2048
 data			   __pycache__	 visualization.ipynb
 datasets		   README.md	 visualization.py
'experiment updating.md'   sampling


In [6]:
!cd ../data

/bin/bash: line 0: cd: ../data: No such file or directory


In [7]:
!cd ..

In [8]:
!cd data

In [9]:
print(os.getcwd())

/home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud


In [11]:
datapath = os.path.join(os.getcwd(), 'data', 'arch')
print(datapath)

/home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/data/arch


In [13]:
import os.path
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
BASE_DIR

NameError: name '__file__' is not defined

In [14]:
!ls

 common.py		   LOG		 shapenetcorev2_hdf5_2048
 data			   __pycache__	 visualization.ipynb
 datasets		   README.md	 visualization.py
'experiment updating.md'   sampling


In [15]:
!cd datasets

In [16]:
print(os.getcwd())

/home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud


In [ ]:
file = "../data/arch/Train/1_TR_cloister.txt"
time3 = datetime.datetime.now()
data = np.loadtxt(file)
scene_points = data[:,0:3].astype('float32')
segment_label = data[:,6].astype('int64')
time4 = datetime.datetime.now()
print(scene_points.shape)
print(segment_label.shape)
print(time4-time3)